# Gem-Flux MCP Server: Session Management

This notebook demonstrates how to manage models and media in the Gem-Flux MCP Server session.

## Session Overview

The Gem-Flux MCP Server uses **session-based storage** (MVP v0.1.0):
- Models and media stored in-memory only
- Storage cleared when server restarts
- No file persistence (future: v0.2.0)

## Available Tools

1. **list_models** - List all models in current session
2. **delete_model** - Remove a model from storage
3. **list_media** - List all media compositions

## Model States

Models have state suffixes indicating processing history:
- `.draft` - Built but not gapfilled
- `.draft.gf` - Draft model after gapfilling
- `.gf` - Gapfilled model (source was already gapfilled)
- `.draft.gf.gf` - Re-gapfilled model

## Setup

In [ ]:
# Import session management tools
from gem_flux_mcp.tools.list_models import list_models
from gem_flux_mcp.tools.delete_model import delete_model
from gem_flux_mcp.tools.list_media import list_media

# Import model building tools for examples
from gem_flux_mcp.tools.build_model import build_model
from gem_flux_mcp.tools.media_builder import build_media
from gem_flux_mcp.tools.gapfill_model import gapfill_model

# Import database and storage modules
from gem_flux_mcp.database.loader import load_compounds_database, load_reactions_database
from gem_flux_mcp.database.index import DatabaseIndex
from gem_flux_mcp.templates.loader import load_templates
from gem_flux_mcp.media.atp_loader import load_atp_media
from gem_flux_mcp.media.predefined_loader import load_predefined_media
from gem_flux_mcp.storage.models import clear_all_models
from gem_flux_mcp.storage.media import clear_all_media

# Import types
from gem_flux_mcp.types import (
    ListModelsRequest,
    DeleteModelRequest,
    ListMediaRequest,
    BuildModelRequest,
    BuildMediaRequest,
    GapfillModelRequest
)

from pathlib import Path
import json

print("✓ Imports successful")

## Initialize Database and Templates

In [ ]:
# Clear session storage
clear_all_models()
clear_all_media()
print("✓ Session storage cleared")

# Load database
database_dir = Path("../data/database")
compounds_path = database_dir / "compounds.tsv"
reactions_path = database_dir / "reactions.tsv"

compounds_df = load_compounds_database(str(compounds_path))
reactions_df = load_reactions_database(str(reactions_path))
db_index = DatabaseIndex(compounds_df, reactions_df)
print(f"✓ Loaded database")

# Load templates
template_dir = Path("../data/templates")
templates = load_templates(str(template_dir))
print(f"✓ Loaded {len(templates)} templates")

# Load ATP media
atp_media = load_atp_media()
print(f"✓ Loaded ATP media")

# Load predefined media
predefined_media = load_predefined_media()
print(f"✓ Loaded {len(predefined_media)} predefined media")

## Part 1: Listing Models

### Empty Session

When the session is empty, list_models returns an empty list.

In [ ]:
# List models in empty session
request = ListModelsRequest()
response = list_models(request)

print("Models in Session:")
print(f"  Total: {response.total_models}")
print(f"  Draft models: {response.models_by_state['draft']}")
print(f"  Gapfilled models: {response.models_by_state['gapfilled']}")

### Create Multiple Models

Let's create several models to demonstrate listing and filtering.

In [ ]:
# Create first model (E. coli)
build_request_1 = BuildModelRequest(
    protein_sequences={
        "prot1": "MKLVINLVGNSGLGKSTFTQRLIN",
        "prot2": "MKQHKAMIVALERFRKEKRDAALL"
    },
    template="GramNegative",
    model_name="E_coli_K12",
    annotate_with_rast=False
)
response_1 = build_model(build_request_1, db_index)
model_id_1 = response_1.model_id
print(f"✓ Created model: {model_id_1}")

# Create second model (B. subtilis)
build_request_2 = BuildModelRequest(
    protein_sequences={
        "prot1": "MSVALERYGIDEVASIGGLVEVNN",
        "prot2": "MGKVIASKLAGNKAPLYRHIADLA"
    },
    template="GramPositive",
    model_name="B_subtilis_168",
    annotate_with_rast=False
)
response_2 = build_model(build_request_2, db_index)
model_id_2 = response_2.model_id
print(f"✓ Created model: {model_id_2}")

# Create third model (auto-generated name)
build_request_3 = BuildModelRequest(
    protein_sequences={
        "prot1": "MAILDSGIHNGIVEGLMTTVHSIT"
    },
    template="Core",
    annotate_with_rast=False
)
response_3 = build_model(build_request_3, db_index)
model_id_3 = response_3.model_id
print(f"✓ Created model: {model_id_3}")

### List All Models

In [ ]:
# List all models
request = ListModelsRequest()
response = list_models(request)

print(f"Models in Session: {response.total_models}\n")

for model in response.models:
    print(f"  ID: {model['model_id']}")
    print(f"    Name: {model['model_name']}")
    print(f"    State: {model['state']}")
    print(f"    Template: {model['template_used']}")
    print(f"    Reactions: {model['num_reactions']}")
    print(f"    Created: {model['created_at']}")
    print()

print(f"Summary:")
print(f"  Draft models: {response.models_by_state['draft']}")
print(f"  Gapfilled models: {response.models_by_state['gapfilled']}")

### Gapfill a Model

Gapfilling creates a new model with a transformed state suffix.

In [ ]:
# Use predefined media
media_id = "glucose_minimal_aerobic"

# Gapfill first model
gapfill_request = GapfillModelRequest(
    model_id=model_id_1,
    media_id=media_id,
    target_growth_rate=0.01,
    gapfill_mode="complete"
)
gapfill_response = gapfill_model(gapfill_request, db_index)
gapfilled_model_id = gapfill_response.model_id

print(f"✓ Gapfilled model: {gapfilled_model_id}")
print(f"  Original: {gapfill_response.original_model_id}")
print(f"  State transformation: .draft → .draft.gf")

### Filter Models by State

Use the filter_state parameter to show only draft or gapfilled models.

In [ ]:
# List only draft models
request = ListModelsRequest(filter_state="draft")
response = list_models(request)

print(f"Draft Models: {response.total_models}\n")
for model in response.models:
    print(f"  {model['model_id']} ({model['state']})")

print()

# List only gapfilled models
request = ListModelsRequest(filter_state="gapfilled")
response = list_models(request)

print(f"Gapfilled Models: {response.total_models}\n")
for model in response.models:
    print(f"  {model['model_id']} ({model['state']})")
    print(f"    Parent: {model.get('parent_model_id', 'N/A')}")

## Part 2: Deleting Models

### Delete a Specific Model

Remove a model from session storage to free memory.

In [ ]:
# Count before deletion
request = ListModelsRequest()
response_before = list_models(request)
print(f"Models before deletion: {response_before.total_models}")

# Delete the auto-generated model
delete_request = DeleteModelRequest(model_id=model_id_3)
delete_response = delete_model(delete_request)

print(f"\n✓ Deleted model: {delete_response.deleted_model_id}")

# Count after deletion
request = ListModelsRequest()
response_after = list_models(request)
print(f"\nModels after deletion: {response_after.total_models}")

# Show remaining models
print("\nRemaining models:")
for model in response_after.models:
    print(f"  {model['model_id']}")

### Error Handling: Delete Nonexistent Model

Attempting to delete a model that doesn't exist returns an error.

In [ ]:
try:
    # Try to delete a nonexistent model
    delete_request = DeleteModelRequest(model_id="nonexistent_model.draft")
    delete_response = delete_model(delete_request)
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print(f"Message: {str(e)}")
    
    # The error includes a list of available models
    print("\nThis error helps you identify the correct model_id.")

## Part 3: Managing Media

### List Predefined Media

The server comes with 4 predefined media compositions.

In [ ]:
# List all media
request = ListMediaRequest()
response = list_media(request)

print(f"Media in Session: {response.total_media}\n")
print(f"Predefined: {response.predefined_media}")
print(f"User-created: {response.user_created_media}\n")

for media in response.media:
    print(f"  ID: {media['media_id']}")
    print(f"    Name: {media['media_name']}")
    print(f"    Compounds: {media['num_compounds']}")
    print(f"    Type: {media['media_type']}")
    print(f"    Predefined: {media['is_predefined']}")
    if media['compounds_preview']:
        print(f"    Preview: {', '.join(media['compounds_preview'])}")
    print()

### Create Custom Media

Create a custom media composition and see it appear in the list.

In [ ]:
# Create custom pyruvate medium
custom_media_request = BuildMediaRequest(
    compounds=[
        "cpd00020",  # Pyruvate (carbon source)
        "cpd00007",  # O2
        "cpd00001",  # H2O
        "cpd00009",  # Phosphate
        "cpd00067",  # H+
    ],
    default_uptake=100.0,
    custom_bounds={
        "cpd00020": (-5.0, 100.0),
        "cpd00007": (-10.0, 100.0)
    }
)
custom_media_response = build_media(custom_media_request, db_index)
custom_media_id = custom_media_response.media_id

print(f"✓ Created custom media: {custom_media_id}")
print(f"  Compounds: {custom_media_response.num_compounds}")
print(f"  Type: {custom_media_response.media_type}")

### List All Media (Including Custom)

Now the list includes both predefined and user-created media.

In [ ]:
# List all media again
request = ListMediaRequest()
response = list_media(request)

print(f"Media in Session: {response.total_media}\n")
print(f"Predefined: {response.predefined_media}")
print(f"User-created: {response.user_created_media}\n")

# Show only user-created media
user_media = [m for m in response.media if not m['is_predefined']]
print(f"User-Created Media: {len(user_media)}\n")
for media in user_media:
    print(f"  {media['media_id']}")
    print(f"    Compounds: {media['num_compounds']}")
    print(f"    Preview: {', '.join(media['compounds_preview'])}")

## Part 4: Model State Transitions

### Demonstrate Complete State History

Create a model and gapfill it multiple times to show state suffix transformations.

In [ ]:
# Build initial model
build_request = BuildModelRequest(
    protein_sequences={"prot1": "MKLVINLVGNSGLGKSTFTQRLIN"},
    template="Core",
    model_name="state_demo",
    annotate_with_rast=False
)
build_response = build_model(build_request, db_index)
current_id = build_response.model_id
print(f"Step 1 - Build: {current_id}")

# First gapfilling
gapfill_request = GapfillModelRequest(
    model_id=current_id,
    media_id="glucose_minimal_aerobic",
    target_growth_rate=0.01,
    gapfill_mode="complete"
)
gapfill_response = gapfill_model(gapfill_request, db_index)
current_id = gapfill_response.model_id
print(f"Step 2 - Gapfill: {current_id}")

# Second gapfilling (different media)
gapfill_request = GapfillModelRequest(
    model_id=current_id,
    media_id="glucose_minimal_anaerobic",
    target_growth_rate=0.01,
    gapfill_mode="complete"
)
gapfill_response = gapfill_model(gapfill_request, db_index)
current_id = gapfill_response.model_id
print(f"Step 3 - Re-gapfill: {current_id}")

# List all versions
print("\nAll Versions in Storage:")
request = ListModelsRequest()
response = list_models(request)

state_demo_models = [m for m in response.models if 'state_demo' in m['model_id']]
for model in state_demo_models:
    print(f"  {model['model_id']} ({model['state']})")
    print(f"    Reactions: {model['num_reactions']}")

## Part 5: Session Lifecycle

### Current Session Summary

In [ ]:
# Get complete session summary
models_request = ListModelsRequest()
models_response = list_models(models_request)

media_request = ListMediaRequest()
media_response = list_media(media_request)

print("═" * 50)
print("         SESSION SUMMARY")
print("═" * 50)

print(f"\nModels: {models_response.total_models}")
print(f"  Draft: {models_response.models_by_state['draft']}")
print(f"  Gapfilled: {models_response.models_by_state['gapfilled']}")

print(f"\nMedia: {media_response.total_media}")
print(f"  Predefined: {media_response.predefined_media}")
print(f"  User-created: {media_response.user_created_media}")

print("\n" + "═" * 50)
print("Note: This storage is session-only (cleared on restart)")
print("      Future v0.2.0 will add file persistence")
print("═" * 50)

## Summary

This notebook demonstrated:

1. **Listing Models**:
   - List all models in session
   - Filter by state (draft vs gapfilled)
   - View model metadata

2. **Deleting Models**:
   - Remove models from storage
   - Error handling for nonexistent models

3. **Managing Media**:
   - List predefined media library
   - Create custom media
   - Distinguish predefined vs user-created

4. **Model State Transitions**:
   - `.draft` → `.draft.gf` (first gapfilling)
   - `.draft.gf` → `.draft.gf.gf` (re-gapfilling)
   - Original models preserved

5. **Session Lifecycle**:
   - In-memory storage (MVP)
   - Cleared on server restart
   - Future: persistent storage

## Key Takeaways

- **Session-based storage**: Models/media lost on restart (MVP)
- **State suffixes**: Track model processing history
- **Original preservation**: Gapfilling creates new models, keeps originals
- **Predefined media**: 4 standard media available immediately

## Best Practices

1. **Use descriptive model names**: Easier to identify in lists
2. **Keep track of model IDs**: Store in variables or notebook state
3. **Delete unused models**: Free memory during long sessions
4. **Use predefined media**: Faster than building custom media

## Next Steps

- Export important models before server restart (future: v0.2.0)
- Organize models by project or organism
- Compare draft vs gapfilled models

See other notebooks:
- `01_basic_workflow.ipynb` - Complete modeling workflow
- `02_database_lookups.ipynb` - Explore compound/reaction databases
- `04_error_handling.ipynb` - Handle common errors